In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import datetime

In [ ]:
interval_time = datetime.time(second=10)

path = "task4_outfiles_4s"
#path = "../data/part4/task2"

num_files = 3
#num_files = 1

In [3]:
def read_txt_file(file_path):
    """
    Reads a text file and returns
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

In [4]:
mcperf_files = [pd.read_csv(f"{path}/mcperf_{i}.txt", skiprows=3, sep='\s+') for i in range(1, num_files + 1)]
jobs_files = [read_txt_file(f"{path}/jobs_{i}.txt") for i in range(1, num_files + 1)]

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_9805/1870524603.py:1: SyntaxWarning: invalid escape sequence '\s'
  mcperf_files = [pd.read_csv(f"{path}/mcperf_{i}.txt", skiprows=3, sep='\s+') for i in range(1, num_files + 1)]


In [5]:
start_times = [datetime.datetime.fromisoformat(jobs_files[i][0].split()[0]) for i in range(num_files)]
end_times = [datetime.datetime.fromisoformat(jobs_files[i][-1].split()[0]) for i in range(num_files)]

In [6]:
def get_runtime(history):

    duration = datetime.datetime.fromisoformat(history[-1].split()[0]) - datetime.datetime.fromisoformat(history[0].split()[0])
    for i in range(1, len(history) - 1, 2):
        duration -= datetime.datetime.fromisoformat(history[i + 1].split()[0]) - datetime.datetime.fromisoformat(history[i].split()[0])
    
    print(history)
    return duration.total_seconds()

In [7]:
total_durations = [(end_times[i] - start_times[i]).total_seconds() for i in range(num_files)]

jobs = ['blackscholes', 'canneal', 'dedup', 'ferret', 'freqmine', 'radix', 'vips']


jobs_timestamps = [{j : [l for l in jobs_files[i] if j in l] for j in jobs} for i in range(num_files)]
jobs_duration = [{j: get_runtime([e for e in h if 'custom' not in e and 'updated' not in e]) for j,h in jobs_timestamps[i].items()} for i in range(num_files)]

['2025-05-14T09:06:19.897139 start blackscholes [3] 1\n', '2025-05-14T09:08:57.776281 end blackscholes\n']
['2025-05-14T09:06:19.661662 start canneal [2] 1\n', '2025-05-14T09:11:16.750761 end canneal\n']
['2025-05-14T09:08:27.885542 start dedup [1] 1\n', '2025-05-14T09:08:34.471145 pause dedup\n', '2025-05-14T09:08:48.996553 unpause dedup\n', '2025-05-14T09:08:58.689430 pause dedup\n', '2025-05-14T09:14:08.667854 unpause dedup\n', '2025-05-14T09:14:27.390267 end dedup\n']
['2025-05-14T09:14:08.887137 start ferret [2] 3\n', '2025-05-14T09:17:24.769780 end ferret\n']
['2025-05-14T09:08:58.429811 start freqmine [3] 3\n', '2025-05-14T09:14:08.633886 end freqmine\n']
['2025-05-14T09:06:19.445363 start radix [1] 1\n', '2025-05-14T09:06:32.345094 pause radix\n', '2025-05-14T09:06:37.247805 unpause radix\n', '2025-05-14T09:06:40.682170 pause radix\n', '2025-05-14T09:07:22.943790 unpause radix\n', '2025-05-14T09:07:25.139043 pause radix\n', '2025-05-14T09:07:31.545390 unpause radix\n', '2025-05

In [8]:
np.mean(total_durations), np.std(total_durations), total_durations

(np.float64(723.5430226666667),
 np.float64(6.672005275584693),
 [727.742728, 714.125702, 728.760638])

In [9]:
pd.DataFrame(jobs_duration).agg(['mean', 'std'])

,blackscholes,canneal,dedup,ferret,freqmine,radix,vips
mean,156.111161,297.473933,34.859155,194.302954,312.604075,50.055519,58.763654
std,3.728049,4.544630,0.670135,2.110207,2.403934,1.144886,2.781014


In [10]:
slo_violations = [int((mcperf_files[i]['p95'] > 800).sum()) for i in range(num_files)]
slo_violations

[8, 7, 6]